In [24]:
import numpy as np
import pandas as pd
import os
from IPython.display import display, HTML


# === 1. Transformation & Adjustment ===

def calculate_transformation_coeffs(src_points: np.ndarray, tgt_points: np.ndarray) -> tuple:
    X_src, Y_src = src_points[:, 0], src_points[:, 1]
    X_tgt, Y_tgt = tgt_points[:, 0], tgt_points[:, 1]

    design_matrix = np.vstack([np.ones_like(X_src), X_src, Y_src]).T
    A_coeffs, _, _, _ = np.linalg.lstsq(design_matrix, X_tgt, rcond=None)
    B_coeffs, _, _, _ = np.linalg.lstsq(design_matrix, Y_tgt, rcond=None)

    return (*A_coeffs, *B_coeffs)

def apply_transformation(src_points: np.ndarray, A0: float, A1: float, A2: float, B0: float, B1: float, B2: float) -> np.ndarray:
    X, Y, Z = src_points[:, 0], src_points[:, 1], src_points[:, 2]
    Xp = A0 + A1 * X + A2 * Y
    Yp = B0 + B1 * X + B2 * Y
    Zp = Z  # Z remains unchanged
    return np.stack((Xp, Yp, Zp), axis=1)

# === 2. Reporting ===

def format_transformation_report(A0, A1, A2, B0, B1, B2, src_points, tgt_points, residuals, z_warning=False) -> str:
    report = []

    if z_warning:
        report.append("WARNING: Z and Z' are not equal. Consider using a 3D affine transformation.\n")

    report += [
        "Linear Affine Transformation (2D)",
        "---------------------------------",
        "This transformation maps source coordinates (X, Y) to target coordinates (X', Y') using:",
        "X' = A1 * X + A2 * Y + A0",
        "Y' = B1 * X + B2 * Y + B0\n",
        "Transformation Coefficients:",
        f"A0 = {A0:.12f}", f"A1 = {A1:.12f}", f"A2 = {A2:.12f}",
        f"B0 = {B0:.12f}", f"B1 = {B1:.12f}", f"B2 = {B2:.12f}",
        "\nKnown point pairs",
        "------------------------------------------------"
    ]

    for i, (src, tgt) in enumerate(zip(src_points, tgt_points), 1):
        report.append(f"{i:<9} {src[0]:>10.3f}  {src[1]:>10.3f}  {src[2]:>6.3f}")
        report.append(f"      =>  {tgt[0]:>10.3f}  {tgt[1]:>10.3f}  {tgt[2]:>6.3f}")

    report += [
        "\nResiduals",
        "Number          dX        dY        dZ",
        "------------------------------------------------"
    ]

    dX, dY, dZ = residuals[:, 0], residuals[:, 1], residuals[:, 2]
    for i, (dx, dy, dz) in enumerate(residuals, 1):
        report.append(f"{i:<16}{dx:+9.4f}{dy:+10.4f}{dz:+10.4f}")

    avg_mag = np.mean(np.linalg.norm(residuals, axis=1))
    rms = np.sqrt(np.mean(residuals**2))

    report += [
        "------------------------------------------------",
        f"Max\t{np.max(np.abs(dX)):+9.4f}{np.max(np.abs(dY)):>10.4f}{np.max(np.abs(dZ)):>10.4f}",
        f"Avg\t{avg_mag:10.4f}  {avg_mag:8.4f}  {avg_mag:8.4f}",
        f"RMS\t{rms:10.4f}  {rms:8.4f}  {rms:8.4f}"
    ]

    return "\n".join(report)

# === 3. I/O Utilities ===

def load_point_file(file_path: str, label: str) -> np.ndarray:
    try:
        df = pd.read_csv(file_path)

        if not np.issubdtype(df.iloc[0, 0], np.number):
            df = pd.read_csv(file_path, header=1)

        if df.empty:
            raise ValueError(f"{label} file is empty.")
        if df.shape[1] < 3:
            raise ValueError(f"{label} file must have at least 3 columns (X, Y, Z).")

        coords = df.iloc[:, :3].to_numpy()

        if not np.issubdtype(coords.dtype, np.number):
            raise ValueError(f"{label} file must contain numeric values in the first three columns.")

        return coords

    except Exception as e:
        raise FileNotFoundError(f"Error loading {label} file: {e}")

def write_report(report_text: str, directory: str, report_file_name: str) -> None:
    report_path = os.path.join(directory, report_file_name)
    with open(report_path, "w", encoding="utf-8") as f:
        f.write(report_text)
    print(f"\nReport written to: {report_path}")


def display_txt_as_text(directory, report_file_name):
    """Reads a .txt file and displays its contents as plain text."""
    report_path = os.path.join(directory, report_file_name)
    try:
        with open(report_path, 'r', encoding='utf-8') as f:
            content = f.read()
        print(content)  # This ensures plain text display
    except FileNotFoundError:
        print(f"File not found: {report_path}")
    except Exception as e:
        print(f"An error occurred: {e}")


# === 4. Affine Estimation & Reporting Pipeline ===

def estimate_affine_transformation(src_points: np.ndarray, tgt_points: np.ndarray) -> dict:
    A0, A1, A2, B0, B1, B2 = calculate_transformation_coeffs(src_points, tgt_points)
    transformed = apply_transformation(src_points, A0, A1, A2, B0, B1, B2)
    residuals = tgt_points - transformed
    z_warning = not np.allclose(src_points[:, 2], tgt_points[:, 2], atol=1e-6)

    return {
        "A0": A0, "A1": A1, "A2": A2,
        "B0": B0, "B1": B1, "B2": B2,
        "residuals": residuals,
        "z_warning": z_warning,
        "transformed": transformed
    }

def write_affine_report(data_dir: str, src_points: np.ndarray, tgt_points: np.ndarray, results: dict, report_file_name: str) -> None:
    report_text = format_transformation_report(
        results["A0"], results["A1"], results["A2"],
        results["B0"], results["B1"], results["B2"],
        src_points, tgt_points, results["residuals"],
        results["z_warning"]
    )
    write_report(report_text, data_dir, report_file_name)

def run_affine_transformation_pipeline(data_dir: str, src_points_file: str, tgt_points_file: str, report_file_name: str) -> None:
    src_path = os.path.join(data_dir, src_points_file)
    tgt_path = os.path.join(data_dir, tgt_points_file)
    src_points = load_point_file(src_path, "source")
    tgt_points = load_point_file(tgt_path, "target")

    results = estimate_affine_transformation(src_points, tgt_points)

    if results["z_warning"]:
        print("\nWARNING: Z and Z' are not equal. Consider using a 3D affine transformation.")

    write_affine_report(data_dir, src_points, tgt_points, results, report_file_name)
    display_txt_as_text(data_dir, report_file_name)


# === 5. Entry Point Configuration ===
DIRECTORY = r"C:\Users\USFJ139860\WSP O365\Southwest Geomatics - Business Development\Marketing & Presentations\2025\NMDOT CIVIL3D CUSTOM COORDINATE SYSTEMS\SMART1-9\CSV\XYZ_GPS"
SOURCE_POINTS = "NM128_SMART1-9 XYZ SOURCE.csv"
TARGET_POINTS = "NM128_SMART1-9  -TM-SF XYZ TARGET.csv"
REPORT = "2D_Affine_Transformation_Report.txt"

run_affine_transformation_pipeline(DIRECTORY, SOURCE_POINTS, TARGET_POINTS, REPORT)

FileNotFoundError: Error loading source file: [Errno 2] No such file or directory: 'C:\\Users\\USFJ139860\\WSP O365\\Southwest Geomatics - Business Development\\Marketing & Presentations\\2025\\NMDOT CIVIL3D CUSTOM COORDINATE SYSTEMS\\SMART1-9\\CSV\\XYZ_GPS\\NM128_SMART1-9 XYZ SOURCE.csv'

In [25]:
DIRECTORY = r"C:\Users\USFJ139860\WSP O365\Southwest Geomatics - Business Development\Marketing & Presentations\2025\NMDOT CIVIL3D CUSTOM COORDINATE SYSTEMS\SMART1-9\CSV\XYZ"
SOURCE_POINTS = "NM128_SMART1-9  - TM-SF XYZ SOURCE.csv"
TARGET_POINTS = "NM128_SMART1-9 XYZ TARGET.csv"

coefficients = run_affine_transformation_pipeline(DIRECTORY, SOURCE_POINTS, TARGET_POINTS)
update_and_write_custom_wkt(DIRECTORY, coefficients)

TypeError: run_affine_transformation_pipeline() missing 1 required positional argument: 'report_file_name'